In [34]:
import os
# Find the latest version of spark 3.x  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.4.0'
spark_version = 'spark-3.4.2'
os.environ['SPARK_VERSION']=spark_version

# Set Environment Variables
os.environ["JAVA_HOME"] = "/Users/morgan.foge/Desktop/hw22/jdk-21.0.2 2"
os.environ["SPARK_HOME"] = '/morgan.foge/hdp/current/spark2-client'

# Start a SparkSession
import findspark
findspark.init()

In [5]:
# Import packages
from pyspark.sql import SparkSession
from pyspark.sql.functions import year, avg, round
import time

# Create a SparkSession
spark = SparkSession.builder.appName("SparkSQL").getOrCreate()

In [11]:
!pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.0/317.0 MB 1.8 MB/s eta 0:00:0000:0100:01
  Preparing metadata (setup.py) ... done
  Obtaining dependency information for py4j==0.10.9.7 from https://files.pythonhosted.org/packages/10/30/a58b32568f1623aaad7db22aa9eafc4c6c194b429ff35bdc55ca2726da47/py4j-0.10.9.7-py2.py3-none-any.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.5/200.5 kB 6.6 MB/s eta 0:00:00
  Created wheel for pyspark: filename=pyspark-3.5.1-py2.py3-none-any.whl size=317488496 sha256=a2ce5e8e2b26861ff2d66f1fea995b7ab8f136ee36e5c02e7f879de24700fdbd
  Stored in directory: /Users/morgan.foge/Library/Caches/pip/wheels/95/13/41/f7f135ee114175605fb4f0a89e7389f3742aa6c1e1a5bcb657
Successfully built pyspark


In [2]:
# 1. Read in the AWS S3 bucket into a DataFrame.

from pyspark import SparkFiles
url = "https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-classroom/v1.2/22-big-data/home_sales_revised.csv"
spark.sparkContext.addFile(url)
df = spark.read.csv("file://" + SparkFiles.get("home_sales_revised.csv"), sep=",", header=True)
print("Number of rows in the DataFrame:", df.count())



Number of rows in the DataFrame: 33287


In [18]:
# 2. Create a temporary view of the DataFrame.
df.createOrReplaceTempView('home_sales')


In [19]:
# 3. What is the average price for a four bedroom house sold per year, rounded to two decimal places?
four_bed_filter = df.filter(df.bedrooms == 4)
avg_yearly = four_bed_filter.groupBy(year("date").alias("year")) \
    .agg(round(avg("price"), 2).alias("avg_price")) \
    .orderBy("year")
avg_yearly.show()


+----+---------+
|year|avg_price|
+----+---------+
|2019| 300263.7|
|2020|298353.78|
|2021|301819.44|
|2022|296363.88|
+----+---------+



In [20]:
# 4. What is the average price of a home for each year the home was built,
# that have 3 bedrooms and 3 bathrooms, rounded to two decimal places?
three_bed_filter = df.filter((df.bedrooms == 3) & (df.bathrooms == 3))
# Calculate the average price per date built
avg_price_three_bed = three_bed_filter.groupBy("date_built") \
    .agg(round(avg("price"), 2).alias("avg_price")) \
    .orderBy("date_built")
avg_price_three_bed.show()


+----------+---------+
|date_built|avg_price|
+----------+---------+
|      2010|292859.62|
|      2011|291117.47|
|      2012|293683.19|
|      2013|295962.27|
|      2014|290852.27|
|      2015| 288770.3|
|      2016|290555.07|
|      2017|292676.79|
+----------+---------+



In [21]:
# 5. What is the average price of a home for each year the home was built,
# that have 3 bedrooms, 3 bathrooms, with two floors,
# and are greater than or equal to 2,000 square feet, rounded to two decimal places?
family_house_filter = df.filter(
    (df.bedrooms == 3) & 
    (df.bathrooms == 3) & 
    (df.floors == 2) & 
    (df.sqft_living >= 2000)
)
# Calculate the average price per date built
avg_price_by_date_built = family_house_filter.groupBy("date_built") \
    .agg(round(avg("price"), 2).alias("avg_price")) \
    .orderBy("date_built")
avg_price_by_date_built.show()


+----------+---------+
|date_built|avg_price|
+----------+---------+
|      2010|285010.22|
|      2011|276553.81|
|      2012|307539.97|
|      2013|303676.79|
|      2014|298264.72|
|      2015|297609.97|
|      2016| 293965.1|
|      2017|280317.58|
+----------+---------+



In [22]:
# 6. What is the average price of a home per "view" rating, rounded to two decimal places,
# having an average home price greater than or equal to $350,000? Order by descending view rating. 
# Although this is a small dataset, determine the run time for this query.

start_time = time.time()
mansion_filter = df.filter(df.price >= 350000)
avg_price_by_view = mansion_filter.groupBy("view") \
    .agg(round(avg("price"), 2).alias("avg_price")) \
    .orderBy("view")
avg_price_by_view.show()
print("--- %s seconds ---" % (time.time() - start_time))

+----+---------+
|view|avg_price|
+----+---------+
|   0|403848.51|
|   1|401044.25|
|  10|401868.43|
| 100|1026669.5|
|  11|399548.12|
|  12|401501.32|
|  13|398917.98|
|  14|398570.03|
|  15| 404673.3|
|  16|399586.53|
|  17|398474.49|
|  18|399332.91|
|  19|398953.17|
|   2|397389.25|
|  20|399522.81|
|  21|399758.88|
|  22|402022.68|
|  23|403411.92|
|  24|400284.92|
|  25|401298.69|
+----+---------+
only showing top 20 rows

--- 0.7653758525848389 seconds ---


In [23]:
# 7. Cache the the temporary table home_sales.
spark.catalog.isCached('home_sales')

False

In [24]:
# 8. Check if the table is cached.
spark.catalog.isCached('home_sales')

False

In [26]:
# 9. Using the cached data, run the last query above, that calculates 
# the average price of a home per "view" rating, rounded to two decimal places,
# having an average home price greater than or equal to $350,000. 
# Determine the runtime and compare it to the uncached runtime.

start_time = time.time()
avg_mansion_filter = df.filter(df.price >= 350000)
avg_view_price = avg_mansion_filter.groupBy("view") \
    .agg(round(avg("price"), 2).alias("avg_price")) \
    .orderBy("view")
avg_mansion_filter.show()
print("--- %s seconds ---" % (time.time() - start_time))


+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|                  id|      date|date_built| price|bedrooms|bathrooms|sqft_living|sqft_lot|floors|waterfront|view|
+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|f8a53099-ba1c-47d...|2022-04-08|      2016|936923|       4|        3|       3167|   11733|     2|         1|  76|
|7530a2d8-1ae3-451...|2021-06-13|      2013|379628|       2|        2|       2235|   14384|     1|         0|  23|
|43de979c-0bf0-4c9...|2019-04-12|      2014|417866|       2|        2|       2127|   10575|     2|         0|   0|
|e0726d4d-d595-407...|2022-01-08|      2017|424418|       3|        2|       2249|   13878|     2|         0|   4|
|131492a1-72e2-4a8...|2020-02-08|      2017|419199|       2|        3|       2062|    8876|     2|         0|   6|
|f1e4cef7-d151-439...|2019-02-01|      2011|398667|       2|        3|       233

In [27]:

OUTPUT_PATH = "p_sales"

In [28]:
# 10. Partition by the "date_built" field on the formatted parquet home sales data 
df.write.partitionBy("date_built").mode("overwrite").parquet(OUTPUT_PATH)

In [29]:
# 11. Read the parquet formatted data.
formatted_parquet=spark.read.parquet(OUTPUT_PATH)

In [30]:
# 12. Create a temporary table for the parquet data.
formatted_parquet.createOrReplaceTempView('p_sales_view')

In [31]:
# 13. Using the parquet DataFrame, run the last query above, that calculates 
# the average price of a home per "view" rating, rounded to two decimal places,
# having an average home price greater than or equal to $350,000. 
# Determine the runtime and compare it to the cached runtime.

start_time = time.time()
any_house_filter = formatted_parquet.filter(formatted_parquet.price >= 350000)
avg_house_price = any_house_filter.groupBy("view") \
    .agg(round(avg("price"), 2).alias("avg_price")) \
    .orderBy("view")
avg_house_price.show()


print("--- %s seconds ---" % (time.time() - start_time))

+----+---------+
|view|avg_price|
+----+---------+
|   0|403848.51|
|   1|401044.25|
|  10|401868.43|
| 100|1026669.5|
|  11|399548.12|
|  12|401501.32|
|  13|398917.98|
|  14|398570.03|
|  15| 404673.3|
|  16|399586.53|
|  17|398474.49|
|  18|399332.91|
|  19|398953.17|
|   2|397389.25|
|  20|399522.81|
|  21|399758.88|
|  22|402022.68|
|  23|403411.92|
|  24|400284.92|
|  25|401298.69|
+----+---------+
only showing top 20 rows

--- 1.225942850112915 seconds ---


In [33]:
# 14. Uncache the home_sales temporary table.
spark.sql("uncache table sales")

DataFrame[]

In [35]:
# 15. Check if the home_sales is no longer cached
if spark.catalog.isCached("sales"):
    print("A table is still is cached")
else:
    # If the DataFrame is not cached, print a message indicating that all is clear
    print("is no longer cached")


is no longer cached
